# Audio Data Block

by @ste & @zachcaceres

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#Export

#Internal dependencies
from exp.nb_AudioCommon import * 

In [3]:
#Export

#External dependencies
import mimetypes
from fastai.vision import *
import torchaudio
from torchaudio import transforms

#for jupyter Display
from IPython.display import Audio

## Sample data for test

In [4]:
# Standard path notation for fast.ai
# The files willbe saved on $HOME/.fastai/data/timit/
path = getFastAiWorkingDirectory('timit')

Working directory: /home/ste/.fastai/data/timit


## Data Block classes

### AudioItem
This is the base class of fast.ai Audio tht holds a reference to and AudioData object.

In [5]:
#Export        
class AudioItem(ItemBase):
    def __init__(self, data:AudioData, **kwargs):
        self.data = data # Always flatten out to single dimension signal!
        self.kwargs = kwargs

    def __str__(self): return f'Duration: {self.duration} seconds.'
    def __len__(self): return self.data.sig.shape[0]
    def _repr_html_(self): return f'{self.__str__()}<br />{self.ipy_audio._repr_html_()}'
    
    def show(self, title:Optional[str]=None, **kwargs):
        "Show sound on `ax` with `title`, using `cmap` if single-channel, overlaid with optional `y`"
        self.hear(title=title)

    def hear(self, title=None):
        if title is not None: print(title)
        display(self.ipy_audio)

    def apply_tfms(self, tfms):
        for tfm in tfms:
            self.data = tfm(self.data)
        return self
        
    @property
    def shape(self):
        return self.data.sig.shape
    
    @property
    def ipy_audio(self): 
        return Audio(data=self.data.sig, rate=self.data.sr)

    @property
    def duration(self): return len(self.data.sig)/self.data.sr

In [6]:
def test_AudioItem_create_from_data():
    signal,samplerate = torchaudio.load(str(path/'TRAIN/DR1/MDPK0/SA1.WAV'))
    a = AudioItem(AudioData(signal,samplerate))
    assert 1 == len(a.data.sig.shape), 'Single dimension data'
    assert a.data.sig.shape[0] > 100, 'Has data'
    assert 16000 == a.data.sr
    display(a)

test_AudioItem_create_from_data()

AudioItem Duration: 2.88 seconds.

In [7]:
def test_AudioItem_create_from_audio_file():
    a = AudioItem(AudioData.load(str(path/'TRAIN/DR1/MDPK0/SA1.WAV')))
    assert 1 == len(a.data.sig.shape), 'Single dimension data'
    assert a.data.sig.shape[0] > 100, 'Has data'
    assert 16000 == a.data.sr
    display(a)
    
test_AudioItem_create_from_audio_file()

NameError: name 'torchaudio' is not defined

## AudioDataBunch

In [40]:
#Export
class AudioDataBunch(DataBunch):
    def hear_ex(self, rows:int=3, ds_type:DatasetType=DatasetType.Valid, **kwargs):
        batch = self.dl(ds_type).dataset[:rows]
        self.train_ds.hear_xys(batch.x, batch.y, **kwargs)

### AudioList
This class is responsible to contain a list of AudioItem.

In [45]:
#Export
class AudioList(ItemList):
    _bunch = AudioDataBunch
    
    # TODO: __REPR__    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
            
    def get(self, i):
        item = self.items[i]
        if isinstance(item, str):
            return AudioItem(AudioData.load(item))
        if isinstance(item, tuple): #data,sr
            return AudioItem(AudioData(item[0],item[1]))
        raise 'Format not supported!'
    
    def reconstruct(self, t:Tensor): return Image(t.transpose(1,2))

    def hear_xys(self, xs, ys, **kwargs):
        for x, y in zip(xs, ys): x.hear(title=y, **kwargs)
            
    @classmethod
    def from_folder(cls, path:PathOrStr='.', extensions:Collection[str]=None, **kwargs)->ItemList:
        extensions = ifnone(extensions, AUDIO_EXTENSIONS)
        return super().from_folder(path=path, extensions=extensions, **kwargs)


In [49]:
def test_AudioList_from_df_file_names():
    import glob
    #Create Data Frame
    df = pd.DataFrame(glob.glob(str(path/'**/*.WAV'), recursive=True)[:10])
    df.columns = ['FileName']
    display(df.head())

    #Crete AudioList
    ils = AudioList.from_df(df, path, cols=['FileName'])
    
    #Test a item
    i=5
    print(f'FileName: {df.FileName[i]}')
    a = ils.get(i)
    print(a.data.sig.shape, a.data.sr)
    display(a)

test_AudioList_from_df_file_names()

,FileName
0,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...
1,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...
2,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...
3,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...
4,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...


FileName: /home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/SX136.WAV
torch.Size([55296]) 16000


AudioItem Duration: 3.456 seconds.

In [50]:
def test_AudioList_from_df_data_and_sr():
    import glob
    #Create Data Frame
    df = pd.DataFrame(glob.glob(str(path/'**/*.WAV'), recursive=True)[:10])
    df.columns = ['FileName']
    df['SampleAndSr']=df['FileName'].apply(lambda n: torchaudio.load(n))
    #df = df['tmp'].drop()
    
    display(df.head())

    #Crete AudioList
    ils = AudioList.from_df(df, path, cols=['SampleAndSr'])
    
    #Test a item
    i=4
    print(f'FileName: {df.FileName[i]}')
    a = ils.get(i)
    print(a.data.sig.shape, a.data.sr)
    display(a)

test_AudioList_from_df_data_and_sr()

,FileName,SampleAndSr
0,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...,"([[tensor(3.0518e-05), tensor(0.), tensor(3.05..."
1,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...,"([[tensor(6.1035e-05), tensor(0.0001), tensor(..."
2,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...,"([[tensor(6.1035e-05), tensor(0.0001), tensor(..."
3,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...,"([[tensor(3.0518e-05), tensor(0.0001), tensor(..."
4,/home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/S...,"([[tensor(3.0518e-05), tensor(9.1553e-05), ten..."


FileName: /home/ste/.fastai/data/timit/TRAIN/DR7/MPAR0/SX46.WAV
torch.Size([30413]) 16000


AudioItem Duration: 1.9008125 seconds.

# ---- TODO: CONTINUE FROM HERE ----

# Audio transformations

In [ ]:
def tfm_to_mel(x):
    """Transform AudioItem to spectrogram"""
    src = x.reshape(1,-1) # expect single sample...
    return transforms.MelSpectrogram(
        sr=16000,
        n_fft=1024,
#        hop_length=512, 
#        n_mels=128,                  
#        power=1.0, 
#        fmin=20, fmax=8000        
    )(src)

In [ ]:
def test_tfm_to_mel():
    sig,sr = torchaudio.load(str(path/'TRAIN/DR1/MDPK0/SA1.WAV'))
    x = AudioItem(sig, sr=sr)
    display(x)
    img = tfm_to_mel(x)
    print(img.shape)
    plt.imshow(img[0])
    
test_tfm_to_mel()

# from_df

In [ ]:
def pad_to_max(t, mx=1000, value=0):
    """Pad tensor with `value` until it reaches length `mx`"""
    if t.shape[1] == mx: return t
    return F.pad(t, (0,0, 0,mx-t.shape[1]), value=value)

In [ ]:
def process_phn_file(p_file, sig, sr, delimiter=' '):
    df = pd.read_csv(p_file, delimiter=delimiter, header=None)
    df.columns = ['Start', 'End', 'Phn']
    df['SampleAndSr'] = df.apply(lambda x : (sig[-1][x['Start']: x['End']], sr), axis=1)
    return df

In [ ]:
import glob 

def create_phn_df(path, count=100):
    phns = []
    final = pd.DataFrame()

    for phn_file in glob.glob(str(path/'**/*.PHN'), recursive=True)[:count]:
        sig,sr = torchaudio.load(str(phn_file.replace('PHN', 'WAV')))
        df = process_phn_file(phn_file, sig, sr, delimiter=' ')
        df['Source'] = phn_file
        final = final.append(df, ignore_index=True)
    return df

df = create_phn_df(path/'TRAIN')
df.head()

In [ ]:
type(df.SampleAndSr[0])

In [ ]:
def tfm_log(x):
    '''Fake transformation that logs x shape'''
    print(f'Shape of transform input: {x.shape}')
    return x

In [ ]:
def tfm_flatten(x):
    return x.reshape(-1)

In [ ]:
# Normal datablock setup from our AudioList from above.
data_from_df = (AudioList.from_df(df, path, cols=['SampleAndSr'])
    .split_by_rand_pct(0.1, seed=1)
    .label_from_df('Phn')
    .transform([[tfm_log, tfm_to_mel, pad_to_max, tfm_log], 
                [tfm_log, tfm_to_mel, pad_to_max, tfm_log]])
    .databunch(bs=8))        

In [ ]:
data_from_df.one_batch(1)

In [ ]:
t0 = AudioList.from_df(df, path, cols=['Sample']); print(type(t0));
#t1 = t0.audio_transform([[tfm_log],[tfm_log]]); print(type(t1));
t1.get(1)

In [ ]:
len(data_from_df.train_dl)

In [ ]:
data_from_df.get(0)

# from_folder

# <span style="color:red">TODO: merge with Yes/No sample</span>

In [ ]:
ils = AudioList.from_folder(path,extensions=AUDIO_EXTENSIONS)

# Export

In [51]:
!python notebook2script.py DataBlock.ipynb

Converted DataBlock.ipynb to nb_DataBlock.py
